In [86]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [87]:
shadow = 'shadow10_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
sigo_files = glob.glob(evaluation_path + "gsigo/*")
sigo_file_list = []
for file in sigo_files:
    sigo_file_list.append(file)

# print('receive log')
# print(sigo_file_list)

In [88]:
shadow = 'shadow10_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
lsgo_files = glob.glob(evaluation_path + "glsgo/*")
lsgo_file_list = []
for file in lsgo_files:
    lsgo_file_list.append(file)

# print('receive log')
# print(lsgo_file_list)

# seed値リストを登録

In [89]:
seed_index = 2
node_index = 3
seed_list = []
nodenum_list = ['200', '300', '400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')

for path in sigo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[seed_index] not in seed_list:
            seed_list.append(match[seed_index])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '10021', '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049']
simulation 試行回数 49


In [90]:
sigo_dict = {}
check_seed = seed_list[0]
sigo_dict[check_seed] = {}
for path in sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        sigo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        sigo_dict[match[seed_index]] = {}
        sigo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新

print(sigo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow10_2/gsigo\\gsigo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/gsigo\\

In [91]:
lsgo_dict = {}
check_seed = seed_list[0]
lsgo_dict[check_seed] = {}
for path in lsgo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        lsgo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        lsgo_dict[match[seed_index]] = {}
        lsgo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
print(lsgo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow10_2/glsgo\\glsgo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow10_2/glsgo\\

In [92]:
# 列ごとの平均を返す関数
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [93]:
# 列ごとの平均を返す関数
def col_average_zero_delete(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [94]:
# packet delivery rationの平均を計算 SIGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        pdr = sigo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

print(total_pdr_list)
print(col_average(total_pdr_list))

[[0.4674867, 0.4, 0.8, '10001'], [0.3674867, 0.5807692, 0.9, '10002'], [0.2529412, 0.3, 0.9, '10003'], [0.3545455, 0.6, 1.0, '10004'], [0.4, 0.4, 0.9, '10005'], [0.36, 0.4, 0.9, '10006'], [0.4058824, 0.6, 1.0, '10007'], [0.35294119999999995, 0.4, 0.8, '10008'], [0.48823530000000004, 0.5, 0.8, '10009'], [0.4, 0.4, 0.7, '10010'], [0.5145455, 0.6799999999999999, 0.7, '10011'], [0.6129412000000001, 0.45517240000000003, 0.9, '10012'], [0.1545455, 0.6448276, 0.7, '10013'], [0.3058824, 0.5, 1.0, '10014'], [0.4529412, 0.4, 0.6, '10015'], [0.3545455, 0.5, 0.8444444000000001, '10016'], [0.2, 0.3, 0.8525, '10017'], [0.3, 0.5, 0.9, '10018'], [0.36, 0.5, 0.8, '10019'], [0.4, 0.7, 0.6, '10020'], [0.2529412, 0.4, 0.9, '10021'], [0.45454550000000005, 0.8, 0.8525, '10022'], [0.3, 0.5, 0.9, '10023'], [0.5, 0.3, 0.8, '10024'], [0.2, 0.5448276, 0.9, '10025'], [0.3545455, 0.6, 0.9, '10026'], [0.4, 0.7, 0.7, '10027'], [0.3, 0.3, 0.7525000000000001, '10028'], [0.2529412, 0.4, 0.7, '10029'], [0.60588239999999

In [95]:
# packet delivery rationの平均を計算 LSGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        pdr = lsgo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

print(total_pdr_list)
print(col_average(total_pdr_list))

[[0.4529412, 0.4448276, 0.7525000000000001, '10001'], [0.2629412, 0.5255968, 0.9, '10002'], [0.1529412, 0.5, 0.7, '10003'], [0.5545455, 0.6807692, 0.7, '10004'], [0.43529409999999996, 0.4, 0.8, '10005'], [0.1, 0.4, 0.7, '10006'], [0.4611765, 0.3, 0.6, '10007'], [0.40748670000000003, 0.3, 0.6, '10008'], [0.3, 0.5, 0.6, '10009'], [0.5552941, 0.5, 0.2, '10010'], [0.3, 0.5, 0.6, '10011'], [0.36, 0.5255968, 0.7, '10012'], [0.2, 0.3, 0.9, '10013'], [0.2682353, 0.3448276, 0.5, '10014'], [0.35294119999999995, 0.3, 0.9, '10015'], [0.3, 0.4, 0.6, '10016'], [0.4052941, 0.4448276, 0.7, '10017'], [0.3, 0.58, 0.6, '10018'], [0.2145455, 0.6, 0.7, '10019'], [0.2, 0.4448276, 0.8, '10020'], [0.43294119999999997, 0.5, 0.8525, '10021'], [0.38, 0.3, 0.8, '10022'], [0.22000000000000003, 0.4, 0.7, '10023'], [0.3, 0.2, 0.7, '10024'], [0.4145455, 0.4448276, 0.7, '10025'], [0.5545455, 0.5, 0.7525000000000001, '10026'], [0.1, 0.6359416, 0.7, '10027'], [0.3, 0.4, 0.6, '10028'], [0.2529412, 0.4, 0.8, '10029'], [0.

In [96]:
# Overheadの平均を計算 SIGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        mr_recvs = sigo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = sigo_df[(sigo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

[1.65919694 1.7565938  1.82727641]


In [97]:
# Overheadの平均を計算 LSGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        mr_recvs = lsgo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = lsgo_df[(lsgo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

[3.06036384 2.46147441 2.36483378]


In [17]:
test_path = sigo_dict['10001']['200']
test_df = pd.read_csv(test_path)
print(test_df['mr_recvs'].sum()) #mr_recvsの合計

#pdr が 0ではないmr_recvの合計
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].sum()) 

#pdrが0ではない　もの(packet)だけをカウント
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].count())

31
31
2


'../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10001nodenum_300.csv'